In [59]:
4

4

In [60]:
%load_ext autoreload
%autoreload 2

import argparse
import os
import copy
import sys
import time
from datetime import datetime
import torch
import random
import numpy as np

from typing import Tuple
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader

import sys, os
sys.path.append(os.getcwd())
# sys.path.append(os.path.join(os.getcwd(), '../'))
from common.utils import HDF5Dataset, DataCreator
from experiments.models_cnn import CNN, ResNet, BasicBlock1d
from experiments.models_fno import FNO1d
from experiments.train_helper import *
from equations.PDEs import PDE, KdV, KS, Heat
from common.augmentation import Subalgebra
from experiments.train import parse_options as lpsda_parse_options, main as lpsda_main

import os, sys
sys.path.append(os.path.join(os.getcwd(), '../sympde'))
from data.dataset import PDEDataset, PDEDataModule
from data.utils import d_to_LT

from model.setup import setup_model
from model.networks.fno import FNO1d
from model.learner import Learner

from viz.plot_pde_data import plot_1ds

from run import parse_options, main

from data.utils import d_to_coords
from data.pde_data_aug import augment_pde1, augment_KdV


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### LPSDA

In [63]:
args_lpsda = lpsda_parse_options(notebook = True)
args_lpsda.train_samples=10
args_lpsda.suffix = 'easy'
args_lpsda.device = 'cpu'

ret_lpsda = lpsda_main(args_lpsda)

dataloader shapes 1 1 1
Training on dataset data/KdV_train_10_easy.h5
models/FNO1d_KdV_samples10_augmentation0000_shiftfourier_future20_time11271052.pt
Number of parameters: 10856084


In [64]:
## CHECK!
ret_lpsda['train_dataset'][0]

(tensor([[0.7238, 0.7124, 0.6985,  ..., 0.7437, 0.7393, 0.7327],
         [0.7270, 0.7162, 0.7028,  ..., 0.7453, 0.7414, 0.7354],
         [0.7301, 0.7198, 0.7069,  ..., 0.7468, 0.7434, 0.7380],
         ...,
         [0.2395, 0.2926, 0.3979,  ..., 0.3884, 0.2936, 0.2410],
         [0.2354, 0.2943, 0.4054,  ..., 0.3718, 0.2799, 0.2316],
         [0.2317, 0.2965, 0.4133,  ..., 0.3553, 0.2664, 0.2225]]),
 tensor(0.4624),
 tensor(0.1976))

In [97]:
ret_lpsda

{'train_dataset': <common.utils.HDF5Dataset at 0x150c1755b910>, 'model': FNO1d}

SymPDE

In [85]:
args = parse_options(notebook=True)
args.pde_name = 'KdV'
args.seed = 42
args.version = f'lpsda_merge'
args.train = True
args.log_dir = '../logs'

ret = main(args)

Seed set to 42


Not augmenting KdV!


In [69]:
## CHECK1
ret['train_dataset'][0]


(tensor([[0.7238, 0.7124, 0.6985,  ..., 0.7437, 0.7393, 0.7327],
         [0.7270, 0.7162, 0.7028,  ..., 0.7453, 0.7414, 0.7354],
         [0.7301, 0.7198, 0.7069,  ..., 0.7468, 0.7434, 0.7380],
         ...,
         [0.2395, 0.2926, 0.3979,  ..., 0.3884, 0.2936, 0.2410],
         [0.2354, 0.2943, 0.4054,  ..., 0.3718, 0.2799, 0.2316],
         [0.2317, 0.2965, 0.4133,  ..., 0.3553, 0.2664, 0.2225]]),
 tensor(0.4624),
 tensor(0.1976))

In [91]:
(u, dx, dt) = ret['train_dataset'][0]
us = u.unsqueeze(0)


In [94]:
batch = (us, dx, dt)

In [96]:
ret['model'](batch)

tensor(0.4624) 256


IndexError: too many indices for tensor of dimension 0

In [84]:
re

torch.Size([1, 20, 256])

In [78]:
ret['model']

FNO1d

### Model

In [74]:
# CHECK1
for i in range(3):
    r = ret['train_dataset']
    r_lpsda = ret_lpsda['train_dataset']
    print(r[0][i] == r_lpsda[0][i])


tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])
tensor(True)
tensor(True)
